In [1]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)

%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np

from src.features import build_features

import glob

ANIO_INICIO = 2017
ANIO_TERMINO = 2035

COLUMNAS_POBLACION_INE = [f"{i}" for i in range(ANIO_INICIO, ANIO_TERMINO + 1)]
COLUMNAS_POBLACION_FONASA = ["2018", "2019", "2020", "2021", "2022"]

COMUNAS_SSMO = [
    "Peñalolén",
    "Macul",
    "Ñuñoa",
    "La Reina",
    "Providencia",
    "Las Condes",
    "Vitacura",
    "Lo Barnechea",
    "Isla de Pascua",
]

COMUNAS_SSMC = ["Santiago", "Maipú", "Estación Central", "Cerrillos"]

## Obtener poblacion INE

In [3]:
ruta_base_de_datos = (
    "../data/raw/3.0-planilla_poblacion_total/estimaciones-y-proyecciones-2002-2035-comunas.xlsx"
)

# Lee la base de datos
df = pd.read_excel(ruta_base_de_datos).iloc[:-2]

# Elimina el sufijo 'Poblacion' de las columnas
df.columns = df.columns.str.replace("Poblacion ", "")

# Renombra columna de hombres o mujeres
df = df.rename(columns={"Sexo\n1=Hombre\n2=Mujer": "hombre_mujer"})

# Indica si es adulto o infantil
df["grupo_etario_poblacion"] = np.where(df["Edad"] >= 15, "Adulto", "Infantil")

## Obtener poblacion FONASA

In [4]:
archivos = glob.glob("../data/raw/5.0-poblacion-fonasa/*.csv")
df_fonasa = pd.concat(pd.read_csv(archivo, encoding="latin-1") for archivo in archivos)

df_fonasa["ANO_INFORMACION"] = df_fonasa.MES_INFORMACION.astype(str).str[:4]

df_fonasa["REGION"] = df_fonasa["REGION"].str.upper().str.strip()
df_fonasa["REGION"] = df_fonasa["REGION"].replace(
    {
        "ÑUBLE": "DE ÑUBLE",
        "DEL LIBERTADOR B. O'HIGGINS": "DEL LIBERTADOR GENERAL BERNARDO O'HIGGINS",
    }
)

df_fonasa["SEXO"] = df_fonasa["SEXO"].str.upper().str.strip()
df_fonasa["SERVICIO_SALUD"] = df_fonasa["SERVICIO_SALUD"].str.upper().str.strip()
df_fonasa["COMUNA"] = df_fonasa["COMUNA"].str.upper().str.strip()

df_fonasa["EDAD_TRAMO"] = df_fonasa["EDAD_TRAMO"].replace(
    {"Más de 99 años": "99 años", "S.I.": "-1"}
)
df_fonasa["EDAD_TRAMO"] = df_fonasa["EDAD_TRAMO"].str.split().str[0].astype(int)

## Definición de Estratos a Calcular

En este cuadernillo se quiere calcular la población para distintos Estratos y Grupos Etarios del País. Para esto, se utilizará la base de datos de la estimación de la población del Instituto Nacional de Estadísticas (INE) entre 2017 y 2035.

En primer lugar, se definirán los siguientes estratos a calcular:

1. País
2. Regiones del País
3. SSMO
4. SSMC

### INE

In [5]:
ESTRATOS_A_CALCULAR_INE = {}

# Obtiene estratos a calcular
df_pais = df.copy()
ESTRATOS_A_CALCULAR_INE["Pais"] = df_pais

for region in df_pais["M"].unique():
    df_region = df_pais.query("M == @region").copy()
    ESTRATOS_A_CALCULAR_INE[region] = df_region

df_ssmo = df_pais[df_pais["Nombre Comuna"].isin(COMUNAS_SSMO)]
ESTRATOS_A_CALCULAR_INE["SSMO"] = df_ssmo

df_ssmc = df_pais[df_pais["Nombre Comuna"].isin(COMUNAS_SSMC)]
ESTRATOS_A_CALCULAR_INE["SSMC"] = df_ssmc

### FONASA

In [6]:
# Obtiene estratos a calcular FONASA
ESTRATOS_A_CALCULAR_FONASA = {}

# Estrato pais
df_pais_fonasa = df_fonasa.copy()
ESTRATOS_A_CALCULAR_FONASA["Pais"] = df_pais_fonasa

# Por region
for region in df_pais_fonasa["REGION"].unique():
    df_region_fonasa = df_pais_fonasa.query("REGION == @region").copy()
    ESTRATOS_A_CALCULAR_FONASA[region] = df_region_fonasa

# Obtiene SSMO y SSMC
df_ssmo_fonasa = df_pais_fonasa.query("SERVICIO_SALUD == 'SERVICIO DE SALUD METROPOLITANO ORIENTE'")
df_ssmc_fonasa = df_pais_fonasa.query("SERVICIO_SALUD == 'SERVICIO DE SALUD METROPOLITANO CENTRAL'")

ESTRATOS_A_CALCULAR_FONASA["SSMO"] = df_ssmo_fonasa
ESTRATOS_A_CALCULAR_FONASA["SSMC"] = df_ssmc_fonasa

## Definición de grupos etarios a calcular

En el próximo diccionario se definirán todos los grupos etarios que se quieren calcular para cada estrato previamente definidos.

### INE

In [7]:
# Define todos los grupos etarios a calcular.
query_strings = {
    "todos": "", # Todo el pais
    "hombres": "hombre_mujer == 1", # Hombres
    "mujeres": "hombre_mujer == 2", # Mujeres
    "recien_nacidos_vivos": "Edad == 0", # Recien Nacidos Vivos
    "sobre_15": "Edad > 15", # Sobre 15 anios
    "sobre_20": "Edad > 20", # Sobre 20 anios
    "sobre_45": "Edad > 45", # Sobre 45 anios
    "sobre_60": "Edad > 60", # Sobre 60 anios
    "sobre_65": "Edad > 65", # Sobre 65 anios
    "sobre_75": "Edad > 75", # Sobre 75 anios
    "entre_15_a_24_anios": "Edad >= 15 and Edad <= 24", # Entre 15 y 24 anios
    "entre_25_a_44_anios": "Edad >= 25 and Edad <= 44", # Entre 25 y 44 anios
    "entre_45_a_59_anios": "Edad >= 45 and Edad <= 59", # Entre 45 y 59 anios
    "entre_60_y_70": "Edad >= 60 and Edad <= 70", # Entre 60 y 70 anios
    "entre_60_a_74_anios": "Edad >= 60 and Edad <= 74", # Entre 60 a 74 anios
    "mayores_a_75_anios": "Edad >= 75", # Mayores a 75 anios

}

Finalmente, se calculará la cantidad de población para distintos estratos y grupos etarios.

In [8]:
# Obtiene resultados por estratos y grupos etarios
poblaciones_estratos_calculados_ine = build_features.iterate_queries(
    ESTRATOS_A_CALCULAR_INE, query_strings, COLUMNAS_POBLACION_INE
)
# Une todos los resultados en un unico DataFrame
poblaciones_estratos_calculados_ine = pd.concat(poblaciones_estratos_calculados_ine)
poblaciones_estratos_calculados_ine = poblaciones_estratos_calculados_ine.reset_index(
    names=["Edad Incidencia", "Estrato"]
)

### FONASA

In [9]:
# Define todos los grupos etarios a calcular.
query_strings = {
    "todos": "", # Todo el pais
    "hombres": "SEXO == 'HOMBRE'", # Hombres
    "mujeres": "SEXO == 'MUJER'", # Mujeres
    "recien_nacidos_vivos": "", # Recien Nacidos Vivos
    "sobre_15": "EDAD_TRAMO >= 15", # Sobre 15 anios
    "sobre_20": "EDAD_TRAMO >= 20", # Sobre 20 anios
    "sobre_45": "EDAD_TRAMO >= 45", # Sobre 45 anios
    "sobre_60": "EDAD_TRAMO >= 60", # Sobre 60 anios
    "sobre_65": "EDAD_TRAMO >= 65", # Sobre 65 anios
    "sobre_75": "EDAD_TRAMO >= 75", # Sobre 75 anios
    "entre_15_a_24_anios": "EDAD_TRAMO >= 15 and EDAD_TRAMO < 25", # Entre 15 y 24 anios
    "entre_25_a_44_anios": "EDAD_TRAMO >= 25 and EDAD_TRAMO < 45", # Entre 25 y 44 anios
    "entre_45_a_59_anios": "EDAD_TRAMO >= 45 and EDAD_TRAMO < 60", # Entre 45 y 59 anios
    "entre_60_y_70_anios": "EDAD_TRAMO >= 60 and EDAD_TRAMO <= 70", # Entre 60 y 70 anios
    "entre_60_a_74_anios": "EDAD_TRAMO >= 60 and EDAD_TRAMO < 75", # Entre 60 a 74 anios
    "mayores_a_75_anios": "EDAD_TRAMO >= 75", # Mayores a 75 anios
}

In [10]:
# Obtiene resultados por estratos y grupos etarios
poblaciones_estratos_calculados_fonasa = build_features.iterate_queries(
    ESTRATOS_A_CALCULAR_FONASA, query_strings, "CUENTA_BENEFICIARIOS", "FONASA"
)

# Une todos los resultaods en un unico DataFrame
poblaciones_estratos_calculados_fonasa = pd.concat(poblaciones_estratos_calculados_fonasa)
poblaciones_estratos_calculados_fonasa = poblaciones_estratos_calculados_fonasa.reset_index(
    names=["Edad Incidencia", "Estrato"]
)

## Obtener porcentaje de población FONASA total

En este caso, se quiere saber el % de población FONASA a nivel Nacional. Esto se utilizará para obtener los casos FONASA al 2035.

In [11]:
# Obtiene poblacion INE entre 2018 y 2022
poblacion_INE_nacional_2018_y_2022 = poblaciones_estratos_calculados_ine.query(
    "`Edad Incidencia` == 'todos' and Estrato == 'Pais'"
)[COLUMNAS_POBLACION_FONASA]

# Obtiene poblacion FONASA entre 2018 y 2022
poblacion_FONASA_nacional_2018_y_2022 = poblaciones_estratos_calculados_fonasa.query(
    "`Edad Incidencia` == 'todos' and Estrato == 'Pais'"
)[COLUMNAS_POBLACION_FONASA]

# Obtiene porcentaje de la poblacion FONASA promedio a nivel nacional acumulado entre 2018 y 2022
PORCENTAJE_FONASA_NACIONAL = (
    poblacion_FONASA_nacional_2018_y_2022.sum(axis=1)
    / poblacion_INE_nacional_2018_y_2022.sum(axis=1)
)[0]

print(f"Entre 2018 y 2022, el {PORCENTAJE_FONASA_NACIONAL:.2%} de la poblacion Nacional fue FONASA")

Entre 2018 y 2022, el 77.39% de la poblacion Nacional fue FONASA


---

En este apartado se leerán las incidencias, junto a las poblaciones respectivas donde se tienen que aplicar tales incidencias. Posteriormente, se calcularán los casos teóricos para cada problema de salud, y para cada estrato definido.

## Lectura de Incidencias por problema de salud

In [12]:
RUTA_PLANILLA_INCIDENCIAS = "../data/external/incidencias_y_prevalencias.xlsx"
COLUMNAS_A_UTILIZAR = [
    "Diagnostico",
    "Estadística",
    "Casos (Cada 100.000)",
    "Edad Incidencia",
    "Área de Influencia",
]


# Lee planilla de incidencias
incidencias = pd.read_excel(RUTA_PLANILLA_INCIDENCIAS, usecols=COLUMNAS_A_UTILIZAR)
# Convierte las incidencia a numeros e imputa las faltantes como NaN
incidencias["Casos (Cada 100.000)"] = pd.to_numeric(
    incidencias["Casos (Cada 100.000)"], errors="coerce"
)

# Obtiene la fraccion de la incidencia, para multiplicar con la poblacion directamente
incidencias["rate_incidencia"] = incidencias["Casos (Cada 100.000)"] / 100000

## Estimacion de casos con INE

In [18]:
# Une las incidencias con las poblacion calculadas segun los estratos respectivos
union_poblaciones_e_incidencias_ine = incidencias.merge(
    poblaciones_estratos_calculados_ine, how="left", on="Edad Incidencia"
)

# Multiplica la incidencia por las poblaciones
union_poblaciones_e_incidencias_ine[COLUMNAS_POBLACION_INE] = union_poblaciones_e_incidencias_ine[
    COLUMNAS_POBLACION_INE
].multiply(union_poblaciones_e_incidencias_ine["rate_incidencia"], axis="index")

# # Elimina rate de incidencia para claridad
# union_poblaciones_e_incidencias_ine = union_poblaciones_e_incidencias_ine.drop(
#     columns=["rate_incidencia", "Área de Influencia"]
# )

## Estimacion de casos con FONASA

In [19]:
# Une las incidencias con las poblacion calculadas segun los estratos respectivos
union_poblaciones_e_incidencias_fonasa = incidencias.merge(
    poblaciones_estratos_calculados_fonasa, how="left", on="Edad Incidencia"
)

# Multiplica la incidencia por las poblaciones
union_poblaciones_e_incidencias_fonasa[COLUMNAS_POBLACION_FONASA] = (
    union_poblaciones_e_incidencias_fonasa[COLUMNAS_POBLACION_FONASA].multiply(
        union_poblaciones_e_incidencias_fonasa["rate_incidencia"], axis="index"
    )
)

# # Elimina rate de incidencia para claridad
# union_poblaciones_e_incidencias_fonasa = union_poblaciones_e_incidencias_fonasa.drop(
#     columns=["rate_incidencia", "Área de Influencia"]
# )

## Estimacion de casos FONASA, utilizando la poblacion INE y el % historico de la poblacion FONASA

In [20]:
# Copia los casos INE ya calculados
union_poblaciones_e_incidencias_fonasa_desde_ine = union_poblaciones_e_incidencias_ine.copy()

# Obtiene los casos FONASA, utilizando el % de poblacion FONASA historica del total
union_poblaciones_e_incidencias_fonasa_desde_ine[COLUMNAS_POBLACION_INE] = (
    union_poblaciones_e_incidencias_ine[COLUMNAS_POBLACION_INE] * PORCENTAJE_FONASA_NACIONAL
)


## Acotación de casos según área de influencia para el INT

Una vez obtenido los casos proyectados hasta el 2035, se procederá a acotar cada uno de los diagnósticos y sus casos según su área de influencia determinada.

In [40]:
area_de_infl_INT = union_poblaciones_e_incidencias_fonasa_desde_ine.copy()

# Separa las areas de influencia por comas, para dejarlas en una lista
area_de_infl_INT["Área de Influencia"] = area_de_infl_INT["Área de Influencia"].str.split(",")
area_de_infl_INT = area_de_infl_INT.explode("Área de Influencia")
area_de_infl_INT["Área de Influencia"] = area_de_infl_INT["Área de Influencia"].str.strip()

# Solamente deja las filas donde el Area de Influencia coincida con el Estrato
area_de_infl_INT = area_de_infl_INT.query("`Área de Influencia` == `Estrato`")

In [41]:
area_de_infl_INT

,Diagnostico,Estadística,Casos (Cada 100.000),Edad Incidencia,Área de Influencia,rate_incidencia,Estrato,2017,2018,2019,...,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
17,"A152 - Tuberculosis del pulmón, confirmada his...",Incidencia,13.300,todos,SSMO,0.000133,SSMO,139.736866,143.358314,147.358037,...,158.517281,159.403935,160.262901,161.087178,161.877591,162.642065,163.380704,164.084448,164.756901,165.395181
55,"C341 - Tumor maligno del lóbulo superior, bron...",Incidencia,40.000,todos,SSMO,0.000400,SSMO,420.261252,431.152825,443.182065,...,476.743702,479.410331,481.993686,484.472716,486.849898,489.149068,491.370537,493.487062,495.509477,497.429116
56,"C341 - Tumor maligno del lóbulo superior, bron...",Incidencia,40.000,todos,SSMC,0.000400,SSMC,380.617509,395.383341,411.090573,...,452.892013,455.630153,458.151904,460.441787,462.493921,464.330595,465.937879,467.302460,468.419386,469.282774
74,"C342 - Tumor maligno del lóbulo medio, bronqui...",Incidencia,40.000,todos,SSMO,0.000400,SSMO,420.261252,431.152825,443.182065,...,476.743702,479.410331,481.993686,484.472716,486.849898,489.149068,491.370537,493.487062,495.509477,497.429116
75,"C342 - Tumor maligno del lóbulo medio, bronqui...",Incidencia,40.000,todos,SSMC,0.000400,SSMC,380.617509,395.383341,411.090573,...,452.892013,455.630153,458.151904,460.441787,462.493921,464.330595,465.937879,467.302460,468.419386,469.282774
93,"C343 - Tumor maligno del lóbulo inferior, bron...",Incidencia,40.000,todos,SSMO,0.000400,SSMO,420.261252,431.152825,443.182065,...,476.743702,479.410331,481.993686,484.472716,486.849898,489.149068,491.370537,493.487062,495.509477,497.429116
94,"C343 - Tumor maligno del lóbulo inferior, bron...",Incidencia,40.000,todos,SSMC,0.000400,SSMC,380.617509,395.383341,411.090573,...,452.892013,455.630153,458.151904,460.441787,462.493921,464.330595,465.937879,467.302460,468.419386,469.282774
131,C780 - Tumor maligno secundario del pulmón,Incidencia,40.000,todos,SSMO,0.000400,SSMO,420.261252,431.152825,443.182065,...,476.743702,479.410331,481.993686,484.472716,486.849898,489.149068,491.370537,493.487062,495.509477,497.429116
132,C780 - Tumor maligno secundario del pulmón,Incidencia,40.000,todos,SSMC,0.000400,SSMC,380.617509,395.383341,411.090573,...,452.892013,455.630153,458.151904,460.441787,462.493921,464.330595,465.937879,467.302460,468.419386,469.282774
150,C782 - Tumor maligno secundario de la pleura,Incidencia,40.000,todos,SSMO,0.000400,SSMO,420.261252,431.152825,443.182065,...,476.743702,479.410331,481.993686,484.472716,486.849898,489.149068,491.370537,493.487062,495.509477,497.429116


In [ ]:
with pd.ExcelWriter("../data/interim/casos_teoricos_diagnosticos.xlsx") as writer:
    union_poblaciones_e_incidencias_ine.to_excel(
        writer, sheet_name="casos_teoricos_INE", index=False
    )

    union_poblaciones_e_incidencias_fonasa_desde_ine.to_excel(
        writer, sheet_name="casos_teoricos_FONASA_desde_INE", index=False
    )

    union_poblaciones_e_incidencias_fonasa.to_excel(
        writer, sheet_name="casos_teoricos_FONASA", index=False
    )

    poblaciones_estratos_calculados_ine.to_excel(writer, sheet_name="poblacion_INE", index=False)

    poblaciones_estratos_calculados_fonasa.to_excel(
        writer, sheet_name="poblacion_FONASA", index=False
    )